# Libraries

In [1]:
import os
import dotenv

dotenv.load_dotenv()

True

## Check if the apis are working

In [71]:
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere

# Set model
model = GoogleGenerativeAI(model="gemini-pro").invoke("Hello, how are you?")
print("--Model API Check--")
print(model)
print()

# Set embeddings
embeddings = CohereEmbeddings(model="embed-english-light-v3.0")
text = "This is a test document."
query_result = embeddings.embed_query(text)
print("--Embeddings API Check--")
print(query_result)
print()

# Set web search
web_search_tool = TavilySearchResults()
search_results = web_search_tool.invoke("python")
print("--Web Search API Check--")
print(search_results)
print()

llm = ChatCohere(model="command-r", temperature=0)
print(llm.invoke("What is the capital of France?"))
print()

--Model API Check--
I am well, thank you for asking. How are you doing today?

--Embeddings API Check--
[-0.09338379, 0.0871582, -0.03326416, 0.01953125, 0.07702637, 0.034729004, -0.058380127, -0.031021118, -0.030517578, -0.055999756, 0.050842285, -0.006752014, 0.038391113, -0.0014362335, -0.041137695, -0.008880615, 0.026000977, -0.023010254, 0.05456543, -0.03366089, 0.055633545, 0.028579712, -0.068603516, 0.03970337, -0.06677246, 0.06732178, -0.013053894, -0.0060920715, 0.038116455, 0.057800293, 0.048736572, 0.026855469, 0.009849548, 0.08312988, 0.073791504, 0.01663208, -0.0871582, 0.01802063, -0.0020828247, -0.0031356812, 0.039978027, -0.03164673, 0.009796143, 0.011375427, 0.0068855286, 0.092285156, 0.05218506, -0.060943604, 0.038269043, -0.018218994, -0.04510498, -0.0847168, 0.008300781, -0.060058594, 0.0012111664, 0.05102539, 0.05218506, -0.047210693, -0.051239014, -0.044158936, -0.058166504, 0.07849121, -0.019165039, 0.06451416, 0.024887085, 0.011405945, -0.03768921, -0.018814087,

# Creating a vector store

In [72]:
import os
use_serverless = True

In [73]:
from pinecone import Pinecone, ServerlessSpec, PodSpec  
import time  
# configure client  
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)  
if use_serverless:  
    spec = ServerlessSpec(cloud='aws', region='us-east-1')  
else:  
    # if not using a starter index, you should specify a pod_type too  
    spec = PodSpec()  
# check for and delete index if already exists  
index_name = 'pdf-chatter'  
if index_name in pc.list_indexes().names():  
    pc.delete_index(index_name)  
# create a new index  
pc.create_index(  
    index_name,  
    dimension=1024,  # dimensionality of text-embedding-ada-002  
    metric='cosine',  
    spec=spec  
)  
# wait for index to be initialized  
while not pc.describe_index(index_name).status['ready']:  
    time.sleep(1)

# Setting up vector store of the given pdf

In [74]:
# Loading a PDF document
from langchain_community.document_loaders import PyPDFLoader

pdf = "research.pdf"
pdf_description = "This is a research paper on the topic of AI."

loader = PyPDFLoader(pdf, extract_images=True)
pages = loader.load_and_split()

In [75]:
# Splitting the document into chunks
from langchain_text_splitters import CharacterTextSplitter


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)


In [76]:
docs

[Document(page_content='Backprop-Free Reinforcement Learning with Active Neural Generative Coding\nAlexander G. Ororbia1, Ankur Mali2\n1Rochester Institute of Technology\n2The Pennsylvania State University\nago@cs.rit.edu, aam35@psu.edu\nAbstract\nIn humans, perceptual awareness facilitates the fast recogni-\ntion and extraction of information from sensory input. This\nawareness largely depends on how the human agent interacts\nwith the environment. In this work, we propose active neural\ngenerative coding, a computational framework for learning\naction-driven generative models without backpropagation of\nerrors (backprop) in dynamic environments. Speciﬁcally, we\ndevelop an intelligent agent that operates even with sparse re-\nwards, drawing inspiration from the cognitive theory of plan-\nning as inference. We demonstrate on several simple control\nproblems that our framework performs competitively with\ndeep Q-learning. The robust performance of our agent offers\npromising evidence t

# Retriver

In [77]:
from langchain_pinecone import PineconeVectorStore  
from langchain_cohere import CohereEmbeddings

index_name = "pdf-chatter"
embeddings = CohereEmbeddings(model="embed-multilingual-v3.0")

# To add documents to the Vector DB.
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name,namespace=pdf)
retriever = docsearch.as_retriever()

In [13]:
# # Just to Load from the DB
# index_name = "pdf-chatter"
# pdf = "ex.pdf"
# pdf_description = "This is book of PHARMACOLOGY-II  B.Pharm, Semester-V."
# docsearch = PineconeVectorStore(index_name=index_name, embedding=embeddings, namespace=pdf)
# retriever = docsearch.as_retriever()

## Conversation Memory Component

In [190]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_cohere import ChatCohere

# LLM
llm = ChatCohere(model_name="command-r", temperature=0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1000,return_messages=True)
previous_history = " "


## LLMs

We use a router to pick between tools. 
 
Cohere model decides which tool(s) to call, as well as the how to query them.

## Question Router

In [211]:
### Router
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere

# Data model
class web_search(BaseModel):
    """
    The internet. Use web_search for questions that are related to anything else than agents, prompt engineering, and adversarial attacks.
    """
    query: str = Field(description="The query to use when searching the internet.")

class vectorstore(BaseModel):
    """
    A vectorstore containing documents related to agents, prompt engineering, and adversarial attacks. Use the vectorstore for questions on these topics.
    """
    query: str = Field(description="The query to use when searching the vectorstore.")

# Preamble
preamble = f'''You are a master at finding information! You have two knowledge sources:
        1. A massive document storage system containing detailed summaries ({pdf_description}) (vectorstore).
        2. web_search.
    When a user asks a question, if it directly relates to a document description in the storage system, use vectorstore to answer their question. 
    Otherwise, web_search for the best answer.'''
# LLM with tool use and preamble
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_router = llm.bind_tools(tools=[web_search, vectorstore], preamble=preamble)

# Prompt
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}")
    ]
)

question_router = route_prompt | structured_llm_router
# response = question_router.invoke({"question": "What is the capital of France?"})
# print(response.response_metadata['tool_calls'])
# response = question_router.invoke({"question": "What is the contact number of Tirth Patel?"})
# print(response.response_metadata['tool_calls'])
# response = question_router.invoke({"question": "Hi how are you?"})
# print('tool_calls' in response.response_metadata)

In [212]:
question_router.invoke({"question": "What is the capital of France?"})

{'datasource': 'web_search'}

## Retrieval Grader
Grades every document which is relevant or not based on the question for the input of LLM to answer the question.

In [80]:
### Retrieval Grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# Prompt
preamble = """You are a grader assessing relevance of a retrieved document to a user question. \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# LLM with function call
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments, preamble=preamble)

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question} \n\n Is the document relevant to the user question?"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
# question = "What is Deep Learning?"
# docs = retriever.invoke(question)
# doc_txt = docs[1].page_content
# response =  retrieval_grader.invoke({"question": question, "document": doc_txt})
# print(response)

## Generater LLM
This simply generates answer based on the question given to it.

In [81]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
import langchain
from langchain_core.messages import HumanMessage
from langchain_core.prompts import MessagesPlaceholder


# Preamble
preamble = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."""

# LLM
llm = ChatCohere(model_name="command-r", temperature=0).bind(preamble=preamble)

# Prompt
prompt = lambda x: ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("history"),
        HumanMessage(
            f"Question: {x['question']} \nAnswer: ",
            additional_kwargs={"documents": x["documents"]},
        )
    ]
)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# question = "What is my name?"
# print(question)
# print(docs)
# history = [
#     "My name is Tirth Patel. I am a student at the University of Texas at Dallas. I am studying Computer Science."
# ]
# # Run
# generation = rag_chain.invoke({"documents": docs, "question": question, "history": history})
# print(generation)

## LLM fallback
When question is just a basic conversation than this is used.

In [115]:
### LLM fallback

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
import langchain
from langchain_core.messages import HumanMessage


# Preamble
preamble = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. Use three sentences maximum and keep the answer concise."""

# LLM
llm = ChatCohere(model_name="command-r", temperature=0).bind(preamble=preamble)

# Prompt
prompt = lambda x: ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("history"),
        HumanMessage(
            f"Question: {x['question']} \nAnswer: "
        )
    ]
)

# Chain
llm_chain = prompt | llm | StrOutputParser()

# Run
question = "what is my name"
# history = [
#     " "
# ]
# generation = llm_chain.invoke({"question": question, "history": history})
# print(generation)

## Hallucination Grader
This's basic use is to tell wether the generated answer is a hallucination of the LLM or not.

In [116]:
### Hallucination Grader

# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(description="Answer is grounded in the facts, 'yes' or 'no'")

# Preamble
preamble = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

# LLM with function call
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations, preamble=preamble)

# Prompt
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        # ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
# hallucination_grader.invoke({"documents": docs, "generation": generation})

## Answer Grader
This is the Final check of the answer wether the generated answer serves the asked question of not.

In [84]:
### Answer Grader

# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")

# Preamble
preamble = """You are a grader assessing whether an answer addresses / resolves a question \n
Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""

# LLM with function call
llm = ChatCohere(model="command-r", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer, preamble=preamble)

# Prompt
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
# answer_grader.invoke({"question": question,"generation": generation})

## Web Search Tool

In [85]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults()

# Graph

Capture the flow in as a graph.

## Graph state

In [157]:
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    """|
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        history: list of chat history
    """
    question : str
    generation : str
    documents : List[str]

## Graph Flow

In [180]:
from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def llm_fallback(state):
    """
    Generate answer using the LLM w/o vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---LLM Fallback---")
    question = state["question"]

    generation = llm_chain.invoke({"question": question, "history": [previous_history]})
    return {"question": question, "generation": generation}

def generate(state):
    """
    Generate answer using the vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    if not isinstance(documents, list):
      documents = [documents]

    # RAG generation
    generation = rag_chain.invoke({"documents": documents, "question": question, "history": [previous_history]})
    print(generation)
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}

### Edges ###

def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    
    # Fallback to LLM or raise error if no decision
    if "tool_calls" not in source.additional_kwargs:
        print("---ROUTE QUESTION TO LLM---")
        return "llm_fallback"
    if len(source.additional_kwargs["tool_calls"]) == 0:
      raise "Router could not decide source"

    # Choose datasource
    datasource = source.additional_kwargs["tool_calls"][0]["function"]["name"]
    if datasource == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif datasource == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    else: 
        print("---ROUTE QUESTION TO LLM---")
        return "vectorstore"

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, WEB SEARCH---")
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    print("dlsknflsdn",type(score),score)
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY USING WEBSEARCH---")
        return "not useful"












def save_chat(state):
    """
    Save the chat history

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """
    global previous_history
    print("---SAVE CHAT---")
    question = state["question"]
    generation = state["generation"]
    memory.save_context({"input": question}, {"output": generation})
    # print("previous_history",previous_history)
    previous_history = memory.predict_new_summary(memory.chat_memory.messages, previous_history)
    # print("previous_history",previous_history)
    memory.clear()
    return 


## Build Graph

In [178]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # rag
workflow.add_node("llm_fallback", llm_fallback) # llm
workflow.add_node("save_chat", save_chat) # save chat

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
        "llm_fallback": "llm_fallback",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("save_chat", END)
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not useful": "web_search", # Fails to answer question: fall-back to web-search 
        "useful": "save_chat", # Successfully answers question: save chat
    },
)
workflow.add_edge("llm_fallback", "save_chat")

# Compile
app = workflow.compile()

In [179]:
app.invoke({"question": "How Are you?"})

---ROUTE QUESTION---
---ROUTE QUESTION TO LLM---
---LLM Fallback---
---SAVE CHAT---
previous_history The human asks about the AI's state of being. The AI responds that, while it doesn't have emotions or feelings, it is functioning as intended.


{'question': 'How Are you?',
 'generation': 'I am an AI assistant, and I do not have emotions or feelings, but I am functioning as intended.'}

In [181]:
app.invoke({"question": "my name is tirth remenber it"})

---ROUTE QUESTION---
---ROUTE QUESTION TO LLM---
---LLM Fallback---
---SAVE CHAT---
previous_history The human asks about the AI's state of being and name. The AI responds that it is functioning as intended and without emotions or feelings. It also acknowledges the human's name as Tirth and states that it will remember this for future reference.


{'question': 'my name is tirth remenber it',
 'generation': 'Ok, Tirth. I will remember your name.'}

In [186]:
app.invoke({"question": "What is this given PDF about?"})

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
---GENERATE---
The PDF is an academic paper about backprop-free reinforcement learning with active neural generative coding.
---CHECK HALLUCINATIONS---
dlsknflsdn <class '__main__.GradeHallucinations'> binary_score='yes'
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
---SAVE CHAT---
previous_history The human, Tirth, inquires about the AI's state of being and name, to which the AI confirms it is functioning as intended, without emotions. It pledges to remember Tirth's name for future interactions. Tirth then asks about the content of a PDF, and the AI summarizes it as an academic paper on backprop-free reinforcement lear

{'question': 'What is this given PDF about?',
 'generation': 'The PDF is an academic paper about backprop-free reinforcement learning with active neural generative coding.',
 'documents': [Document(page_content='References\nArulkumaran, K.; Deisenroth, M. P.; Brundage, M.; and\nBharath, A. A. 2017. A brief survey of deep reinforcement\nlearning. arXiv preprint arXiv:1708.05866.\nBerlyne, D. E. 1966. Curiosity and exploration. Science,\n153(3731): 25–33.\nBotvinick, M.; and Toussaint, M. 2012. Planning as infer-\nence. Trends in cognitive sciences, 16(10): 485–488.\nBurda, Y .; Edwards, H.; Pathak, D.; Storkey, A.; Darrell,\nT.; and Efros, A. A. 2018a. Large-scale study of curiosity-\ndriven learning. arXiv preprint arXiv:1808.04355.\nBurda, Y .; Edwards, H.; Storkey, A.; and Klimov, O. 2018b.\nExploration by random network distillation. arXiv preprint\narXiv:1810.12894.\nClark, A. 2015. Surﬁng uncertainty: Prediction, action, and\nthe embodied mind. Oxford University Press.\nFriston, K

In [187]:
# Run
inputs = {"question": "what is Mechanism of Action? also specify the page number."}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---


TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}

Trace:

https://smith.langchain.com/public/623da7bb-84a7-4e53-a63e-7ccd77fb9be5/r

In [47]:
# Run
inputs = {"question": ""}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value ["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
The authors of this book are Dr. K.V. Otari, Dr. Pragnesh Patani, and Dr. Pankaj Mishra.
---CHECK HALLUCINATIONS---
dlsknflsdn <class '__main__.GradeHallucinations'> binary_score='yes'
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('The authors of this book are Dr. K.V. Otari, Dr. Pragnesh Patani, and Dr. '
 'Pankaj Mishra.')


Trace:

https://smith.langchain.com/public/57f3973b-6879-4fbe-ae31-9ae524c3a697/r

In [20]:
# Run
inputs = {"question": "Translate Tirth Patel in Gujarati."}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value ["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, WEB SEARCH---
"Node 'grade_documents':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
તીર્થ પટેલ.
---CHECK HALLUCINATIONS---
dlsknflsdn <class '__main__.GradeHallucinations'> binary_score='yes'
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
'તીર્થ પટેલ.'


# Having a Memory component Is necessary

In [22]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain

In [ ]:
memory = VectorStoreRetrieverMemory()
conversation_chain = ConversationChain(workflow, memory)


# Clean up the vector store

In [70]:
pc.delete_index(index_name)